Importy

In [134]:
import numpy as np
import os
import pandas as pd
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Podłączenie dysku Google i wczytanie danych do zmiennej

In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
base_data = pd.read_csv("beacon.csv")

Mounted at /content/drive


In [23]:
df = pd.read_csv("beacon.csv")
df.columns
display(df)

,Distance A,Distance B,Distance C,Position X,Position Y
0,0.877463,0.768609,1.457215,122,180
1,1.201608,1.031228,1.893498,122,180
2,1.614344,1.098873,2.112560,122,180
3,1.533095,1.123157,2.233098,122,180
4,1.481930,1.114962,2.317804,122,180
...,...,...,...,...,...
245,0.904473,1.056633,1.966141,122,223
246,0.894097,1.067867,1.954488,122,223
247,0.954574,1.079314,1.928066,122,223
248,0.904473,1.087064,1.901409,122,223


Stworzenie kolumny 'Beacon' i dodanie do niej wartości

In [41]:
df['Beacon'] = 0
dfCopy = df['Beacon'].copy()
dfA = pd.DataFrame({'Beacon' : dfCopy})
dfB = pd.DataFrame({'Beacon' : dfCopy})
dfC = pd.DataFrame({'Beacon' : dfCopy})
dfA['Beacon'] = ['A' if x == 0.0 else np.nan for x in df['Distance A']]
dfB['Beacon'] = ['B' if x == 0.0 else np.nan for x in df['Distance B']]
dfC['Beacon'] = ['C' if x == 0.0 else np.nan for x in df['Distance C']]
df.update(dfA)
df.update(dfB)
df.update(dfC)
display(df)

,Distance A,Distance B,Distance C,Position X,Position Y,Beacon
0,0.877463,0.768609,1.457215,122,180,0
1,1.201608,1.031228,1.893498,122,180,0
2,1.614344,1.098873,2.112560,122,180,0
3,1.533095,1.123157,2.233098,122,180,0
4,1.481930,1.114962,2.317804,122,180,0
...,...,...,...,...,...,...
245,0.904473,1.056633,1.966141,122,223,0
246,0.894097,1.067867,1.954488,122,223,0
247,0.954574,1.079314,1.928066,122,223,0
248,0.904473,1.087064,1.901409,122,223,0


Wyodrębnienie zbiorów treningowych

In [59]:
train_data = pd.DataFrame(df.where(df['Beacon']!= 0))
train_data = train_data.dropna()
train_data

,Distance A,Distance B,Distance C,Position X,Position Y,Beacon
9,1.533095,1.174496,0.000000,122.0,180.0,C
11,1.533095,1.174496,0.000000,122.0,180.0,C
12,1.541852,1.171544,0.000000,122.0,180.0,C
39,1.320379,1.052935,0.000000,79.0,180.0,C
86,0.000000,1.330237,2.233098,79.0,180.0,A
88,1.137443,1.310629,0.000000,79.0,180.0,C
91,1.258867,1.201608,0.000000,165.0,180.0,C
124,1.545238,1.075474,0.000000,165.0,180.0,C
132,0.000000,1.063233,1.360459,165.0,180.0,A
200,0.877463,0.877463,0.000000,122.0,223.0,C


In [181]:
classification_data = pd.DataFrame(df.where(df['Beacon']== 0)).dropna().iloc[:,0:3]
classification_data

,Distance A,Distance B,Distance C
0,0.877463,0.768609,1.457215
1,1.201608,1.031228,1.893498
2,1.614344,1.098873,2.112560
3,1.533095,1.123157,2.233098
4,1.481930,1.114962,2.317804
...,...,...,...
245,0.904473,1.056633,1.966141
246,0.894097,1.067867,1.954488
247,0.954574,1.079314,1.928066
248,0.904473,1.087064,1.901409


In [137]:
encoder = LabelEncoder()
y = encoder.fit_transform(train_data.iloc[:,[-1]]) 
X = train_data.iloc[:,0:3]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Distance A,Distance B,Distance C
9,1.533095,1.174496,0.000000
11,1.533095,1.174496,0.000000
12,1.541852,1.171544,0.000000
39,1.320379,1.052935,0.000000
86,0.000000,1.330237,2.233098
88,1.137443,1.310629,0.000000
91,1.258867,1.201608,0.000000
124,1.545238,1.075474,0.000000
132,0.000000,1.063233,1.360459
200,0.877463,0.877463,0.000000


Stworzenie schematu warstw sieci

In [86]:
struct = [{"input": 3, "output": 4, "activation": "relu"},
          {"input": 4, "output": 6, "activation": "relu"},
          {"input": 6, "output": 4, "activation": "relu"},
          {"input": 4, "output": 1, "activation": "sigmoid"}]

Zainicjowanie funkcji aktywacji relu i sigmoidalnej, wraz z ich odpowiednikami dla propagacji wstecznej

In [5]:
def sigmoid(activation):
    return 1/(1+np.exp(-activation))

def relu(activation):
    return np.maximum(0,activation)

def sigmoid_backward(dA, activation):
    sig = sigmoid(activation)
    return dA * sig * (1 - sig)

def relu_backward(dA, activation):
    dZ = np.array(dA, copy = True)
    dZ[activation <= 0] = 0
    return dZ

Zainicjowanie sieci pierwszymi wagami

In [147]:
def initialize_network_weights(nn_struct, seed):
  np.random.seed(seed)
  layers = len(nn_struct)
  network = {}
  
  for idx, layer in enumerate(nn_struct):
    layer_idx = idx + 1
    layer_input_size = layer["input"]
    layer_output_size = layer["output"]
        
    network['Weight' + str(layer_idx)] = np.random.randn(layer_output_size, layer_input_size) * 0.1
    network['bias' + str(layer_idx)] = np.random.randn(layer_output_size, 1) * 0.1
        
  return network

Funkcje propagacji

In [125]:
def propagate(input, weight, bias, activation):
    
  if input.shape[0] != weight.shape[1]:
    input = input.T

  #print(weight.shape)
  #print(input.shape)
  Z = np.dot(weight, input) + bias #Z = Weight*(inputs) + bias - do wyznaczenia potem pochodnej do propagacji wstacznej
    
  if activation is "relu":
    function = relu
  elif activation is "sigmoid":
    function = sigmoid
  else:
    raise Exception('NO')
        
  return function(Z), Z

In [174]:
def back_propagate(dInput, weight, bias, Z_curr, input_prev, activation):
  m = input_prev.shape[1]
    
  if activation is "relu":
      function = relu_backward
  elif activation is "sigmoid":
      function = sigmoid_backward
  else:
      raise Exception('No')
    
  dZ_curr = function(dInput, Z_curr)
  if input_prev.shape[0] != dZ_curr.shape[1]:
    input_prev = input_prev.T
  dWeight = np.dot(dZ_curr, input_prev)/m
  dBias = np.sum(dZ_curr, axis=1, keepdims=True)/m
 # if dZ_curr.shape[0] != weight.shape[1]:
 #   input_prev = input_prev.T
  dInput_prev = np.dot(weight.T, dZ_curr)

  return dInput_prev, dWeight, dBias

In [90]:
def forward_propagation(X, network, nn_strckt):
  propagation_values={}
  input = X
  for idx, layer in enumerate(nn_strckt):
    layer_idx = idx + 1
    previous_input = input     
    activation = layer["activation"]
    weight = network["Weight" + str(layer_idx)]
    bias = network["bias" + str(layer_idx)]
    input, Z_curr = propagate(previous_input, weight, bias, activation)
        
    propagation_values["input" + str(idx)] = previous_input
    propagation_values["Z" + str(layer_idx)] = Z_curr
  return input, propagation_values

In [170]:
def backward_propagation(Y_hat, Y, propagation_values, network, nn_strckt):
  gradient = {}
  m = Y.shape[0]
  #print(m)
  Y = Y.reshape(Y_hat.shape)
   
  dInput = -(np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
  for layer_idx_prev, layer in reversed(list(enumerate(nn_strckt))):
    layer_idx_curr = layer_idx_prev + 1
    activation = layer["activation"]
        
    dI_curr = dInput
        
    input_prev = propagation_values["input" + str(layer_idx_prev)]
    Z_curr = propagation_values["Z" + str(layer_idx_curr)]
    weight = network["Weight" + str(layer_idx_curr)]
    bias = network["bias" + str(layer_idx_curr)]
        
    dInput, dWeight, dBias = back_propagate(dI_curr, weight, bias, Z_curr, input_prev, activation)
        
    gradient["dWeight" + str(layer_idx_curr)] = dWeight
    gradient["dBias" + str(layer_idx_curr)] = dBias

  return gradient

Funkcja trenująca sieć


In [178]:
def train_model(X, y, nn_struckt, epochs, lr):
  network = initialize_network_weights(nn_struckt, 69)
    
  for i in range(epochs):
      Y_hat, propagation_values = forward_propagation(X, network, nn_struckt)
        
      gradient = backward_propagation(Y_hat, y, propagation_values, network, nn_struckt)
      display(network)
      display(gradient)
      #pętla funkcji optymalizacji
      for layer_idx, layer in enumerate(nn_struckt):
        network["Weight" + str(layer_idx+1)] -= lr * gradient["dWeight" + str(layer_idx+1)]        
        network["bias" + str(layer_idx+1)] -= lr * gradient["dBias" + str(layer_idx+1)]
        
  return network

In [179]:
values = train_model(X_train, y_train, struct, 10, 0.00001)

{'Weight1': array([[ 0.09155071, -0.0603542 ,  0.11622952],
        [-0.06014001, -0.15974857,  0.03977261],
        [ 0.12058654,  0.10562991,  0.08525439],
        [ 0.06893914, -0.02247371, -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627352, -0.02741205,  0.13757662, -0.01902331]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.18547314],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.07012101],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

{'dBias1': array([[-1.30579183e-04],
        [-5.40101496e-05],
        [-1.26865256e-02],
        [ 1.79170236e-03]]), 'dBias2': array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.02514934]]), 'dBias3': array([[ 0.        ],
        [-0.17859264],
        [-0.11372905],
        [ 0.        ]]), 'dBias4': array([[-1.29988221]]), 'dWeight1': array([[-1.25887757e-04,  0.00000000e+00, -2.76565536e-04],
        [-5.20696828e-05,  0.00000000e+00, -1.14393012e-04],
        [-1.61975329e-02, -1.42031189e-02,  5.74119971e-04],
        [ 2.27583784e-03,  1.96392649e-03,  0.00000000e+00]]), 'dWeight2': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
     

{'Weight1': array([[ 0.09155071, -0.0603542 ,  0.11622952],
        [-0.06014001, -0.15974857,  0.03977261],
        [ 0.1205867 ,  0.10563005,  0.08525439],
        [ 0.06893912, -0.02247373, -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627352, -0.02741205,  0.13757666, -0.0190233 ]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.18547331],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.0701209 ],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

{'dBias1': array([[-1.30580883e-04],
        [-5.40108535e-05],
        [-1.26864385e-02],
        [ 1.79168915e-03]]), 'dBias2': array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.02514916]]), 'dBias3': array([[ 0.        ],
        [-0.17859225],
        [-0.11373119],
        [ 0.        ]]), 'dBias4': array([[-1.29987881]]), 'dWeight1': array([[-1.25889397e-04,  0.00000000e+00, -2.76569138e-04],
        [-5.20703614e-05,  0.00000000e+00, -1.14394503e-04],
        [-1.61974234e-02, -1.42030274e-02,  5.74127579e-04],
        [ 2.27582106e-03,  1.96391201e-03,  0.00000000e+00]]), 'dWeight2': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
     

{'Weight1': array([[ 0.09155071, -0.0603542 ,  0.11622952],
        [-0.06014001, -0.15974857,  0.03977261],
        [ 0.12058687,  0.10563019,  0.08525438],
        [ 0.0689391 , -0.02247375, -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627352, -0.02741205,  0.13757669, -0.01902329]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.18547348],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.0701208 ],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

...

{'dBias1': array([[-1.30591086e-04],
        [-5.40150762e-05],
        [-1.26859160e-02],
        [ 1.79160989e-03]]), 'dBias2': array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.02514809]]), 'dBias3': array([[ 0.        ],
        [-0.17858995],
        [-0.11374405],
        [ 0.        ]]), 'dBias4': array([[-1.29985845]]), 'dWeight1': array([[-1.25899233e-04,  0.00000000e+00, -2.76590748e-04],
        [-5.20744324e-05,  0.00000000e+00, -1.14403446e-04],
        [-1.61967663e-02, -1.42024783e-02,  5.74173222e-04],
        [ 2.27572038e-03,  1.96382513e-03,  0.00000000e+00]]), 'dWeight2': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
     

{'Weight1': array([[ 0.09155072, -0.0603542 ,  0.11622954],
        [-0.06014001, -0.15974857,  0.03977262],
        [ 0.12058784,  0.10563105,  0.08525434],
        [ 0.06893896, -0.02247386, -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627353, -0.02741205,  0.1375769 , -0.01902321]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.1854745 ],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.07012015],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

{'dBias1': array([[-1.30592787e-04],
        [-5.40157800e-05],
        [-1.26858290e-02],
        [ 1.79159669e-03]]), 'dBias2': array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.02514791]]), 'dBias3': array([[ 0.        ],
        [-0.17858957],
        [-0.1137462 ],
        [ 0.        ]]), 'dBias4': array([[-1.29985506]]), 'dWeight1': array([[-1.25900873e-04,  0.00000000e+00, -2.76594349e-04],
        [-5.20751109e-05,  0.00000000e+00, -1.14404937e-04],
        [-1.61966568e-02, -1.42023868e-02,  5.74180828e-04],
        [ 2.27570361e-03,  1.96381065e-03,  0.00000000e+00]]), 'dWeight2': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
     

{'Weight1': array([[ 0.09155072, -0.0603542 ,  0.11622954],
        [-0.06014001, -0.15974857,  0.03977262],
        [ 0.120588  ,  0.10563119,  0.08525434],
        [ 0.06893894, -0.02247388, -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627353, -0.02741206,  0.13757693, -0.0190232 ]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.18547467],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.07012005],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

{'dBias1': array([[-1.30594487e-04],
        [-5.40164837e-05],
        [-1.26857419e-02],
        [ 1.79158348e-03]]), 'dBias2': array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.02514773]]), 'dBias3': array([[ 0.        ],
        [-0.17858919],
        [-0.11374834],
        [ 0.        ]]), 'dBias4': array([[-1.29985166]]), 'dWeight1': array([[-1.25902512e-04,  0.00000000e+00, -2.76597950e-04],
        [-5.20757893e-05,  0.00000000e+00, -1.14406427e-04],
        [-1.61965473e-02, -1.42022953e-02,  5.74188435e-04],
        [ 2.27568683e-03,  1.96379617e-03,  0.00000000e+00]]), 'dWeight2': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
     

In [180]:
display(values)

{'Weight1': array([[ 0.09155072, -0.0603542 ,  0.11622954],
        [-0.06014001, -0.15974857,  0.03977262],
        [ 0.12058816,  0.10563133,  0.08525433],
        [ 0.06893891, -0.0224739 , -0.05221822]]),
 'Weight2': array([[-0.04974814, -0.19471233, -0.1848655 ,  0.02646603],
        [ 0.09345718, -0.11699643,  0.07396054, -0.11121546],
        [-0.1981927 , -0.11938717,  0.01686264, -0.20986129],
        [ 0.00191553,  0.03021857, -0.09603824,  0.15917631],
        [ 0.03523331, -0.06519459, -0.11620016, -0.22830376],
        [-0.06627354, -0.02741206,  0.13757696, -0.01902318]]),
 'Weight3': array([[-0.1574905 , -0.14153689,  0.09306458,  0.03959322,  0.14219919,
         -0.24687564],
        [ 0.01993751, -0.12184111, -0.07409264,  0.10623384, -0.07141702,
          0.18547484],
        [ 0.15245991, -0.17937536, -0.06275298,  0.01300763,  0.17247536,
         -0.07011994],
        [ 0.01319087, -0.17023307, -0.21057546,  0.07351219, -0.0670598 ,
         -0.0150966 ]]),
 'Wei

Klasyfikacja/przewidywanie wartości punktów

In [192]:
def predict(values, data_row, nn_struckt):
  Y_hat, _ = forward_propagation(data_row, values, nn_struckt)
  return np.argmax(Y_hat)

In [197]:
for row in range(len(classification_data)):
  prediction = predict(values, classification_data.iloc[row], struct)
  print("For values:\n%s got %d" % (classification_data.iloc[row], prediction))

For values:
Distance A    0.877463
Distance B    0.768609
Distance C    1.457215
Name: 0, dtype: float64 got 2
For values:
Distance A    1.201608
Distance B    1.031228
Distance C    1.893498
Name: 1, dtype: float64 got 2
For values:
Distance A    1.614344
Distance B    1.098873
Distance C    2.112560
Name: 2, dtype: float64 got 2
For values:
Distance A    1.533095
Distance B    1.123157
Distance C    2.233098
Name: 3, dtype: float64 got 2
For values:
Distance A    1.481930
Distance B    1.114962
Distance C    2.317804
Name: 4, dtype: float64 got 2
For values:
Distance A    1.513634
Distance B    1.135640
Distance C    2.296293
Name: 5, dtype: float64 got 2
For values:
Distance A    1.517499
Distance B    1.148356
Distance C    2.388172
Name: 6, dtype: float64 got 2
For values:
Distance A    1.533095
Distance B    1.156963
Distance C    2.428797
Name: 7, dtype: float64 got 2
For values:
Distance A    1.489097
Distance B    1.163176
Distance C    2.498214
Name: 8, dtype: float64 got 2
F